In [ ]:
!pip install -q --upgrade pip
!pip install -q fastapi uvicorn pyngrok nest_asyncio
!pip install -q transformers accelerate bitsandbytes sentencepiece safetensors huggingface_hub pypdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.6 MB/s eta 0:00:00


In [ ]:
import torch
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    try:
        print("Device name:", torch.cuda.get_device_name(0))
    except Exception:
        pass
    print(torch.cuda.memory_summary(device=0, abbreviated=True))
else:
    raise SystemError("No CUDA GPU available — enable GPU in Runtime -> Change runtime type -> GPU")


CUDA available: True
Device name: Tesla T4
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Requested memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|--------------------

In [ ]:
from huggingface_hub import login
from getpass import getpass

hf_token = getpass("Paste your Hugging Face token here (or press Enter to skip): ")
if hf_token:
    login(hf_token)
    print("✅ Logged in to Hugging Face")
else:
    print("⚠️ Skipping login (may hit rate limits)")


Paste your Hugging Face token here (or press Enter to skip): ··········
✅ Logged in to Hugging Face


In [ ]:
# # ===============================================
# # 🔹 LOAD MODEL (with DRIVE CACHE)
# # ===============================================
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
# import os

# MODEL_NAME = "varma007ut/Indian_Legal_Assitant"
# MODEL_PATH = "/content/drive/MyDrive/legal_ai/model_cache"

# os.makedirs(MODEL_PATH, exist_ok=True)
# print(f"✅ Using Drive cache directory: {MODEL_PATH}")

# bnb_config = BitsAndBytesConfig(
#     load_in_8bit=True,
#     llm_int8_threshold=6.0,
# )

# # ✅ Model and tokenizer will be downloaded once, then loaded from Drive each time
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False, cache_dir=MODEL_PATH)
# if tokenizer.pad_token_id is None:
#     tokenizer.pad_token = tokenizer.eos_token

# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     quantization_config=bnb_config,
#     device_map="auto",
#     trust_remote_code=True,
#     cache_dir=MODEL_PATH
# )

# print("✅ Model loaded on:", next(model.parameters()).device)



In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

MODEL_NAME = "varma007ut/Indian_Legal_Assitant"
print("Model:", MODEL_NAME)

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_compute_dtype=torch.float16
# )

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
# if tokenizer.pad_token_id is None:
#     tokenizer.pad_token = tokenizer.eos_token

# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     quantization_config=bnb_config,
#     device_map="auto",
#     trust_remote_code=True
# )
# print("✅ Model loaded on:", next(model.parameters()).device)
# 8 bit
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# MODEL_NAME = "varma007ut/Indian_Legal_Assitant"
# print("Model:", MODEL_NAME)

# 🔹 Use 8-bit quantization instead of 4-bit
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,          # ✅ Enable 8-bit quantization
    llm_int8_threshold=6.0,     # (optional) Default value used for outlier handling
    llm_int8_skip_modules=None  # (optional) You can skip some modules if needed
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

print("✅ Model loaded on:", next(model.parameters()).device)
# 16 bit
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM

# MODEL_NAME = "varma007ut/Indian_Legal_Assitant"
# print("Model:", MODEL_NAME)

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
# if tokenizer.pad_token_id is None:
#     tokenizer.pad_token = tokenizer.eos_token

# # 🔹 Load model directly in float16 (no quantization)
# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     torch_dtype=torch.float16,   # ✅ 16-bit precision
#     device_map="auto",
#     trust_remote_code=True
# )

# print("✅ Model loaded on:", next(model.parameters()).device)


Model: varma007ut/Indian_Legal_Assitant


config.json:   0%|          | 0.00/934 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

...

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

✅ Model loaded on: cuda:0


In [ ]:
# from pypdf import PdfReader
# import re, tempfile, os
# from fastapi import FastAPI, UploadFile, Form
# from fastapi.responses import JSONResponse
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import torch

# # -------- PDF text extraction --------
# def extract_pdf_text(pdf_path):
#     try:
#         reader = PdfReader(pdf_path)
#         pages_text = []
#         for page in reader.pages:
#             txt = page.extract_text()
#             if txt:
#                 pages_text.append(txt)
#         return "\n\n".join(pages_text).strip()
#     except Exception as e:
#         print(f"PDF extraction error: {e}")
#         return ""

# # -------- Summarization --------
# def summarize_text(text, level="short"):
#     if not text.strip():
#         return "No text content found to summarize."

#     # mapping = {
#     #     "short": {
#     #         "max_new_tokens": 120,
#     #         "instruction": "Summarize this legal case in 3-4 sentences focusing on key facts, issue, and outcome:"
#     #     },
#     #     "medium": {
#     #         "max_new_tokens": 250,
#     #         "instruction": "Summarize this legal case in 1-2 paragraphs including key facts, legal issues, arguments, and decision:"
#     #     },
#     #     "long": {
#     #         "max_new_tokens": 400,
#     #         "instruction": "Write a detailed summary of this legal case covering facts, legal issues, arguments, judgment, and outcome:"
#     #     },
#     #     "very_long": {
#     #         "max_new_tokens": 1000,
#     #         "instruction": "Write a comprehensive summary of this legal case with all details including facts, timeline, legal issues, arguments, judgment reasoning, and outcome:"
#     #     }
#     # }
#     # mapping = {
#     # "short": {
#     #     "max_new_tokens": 120,
#     #     "instruction": "Summarize this legal case clearly and concisely in 3-4 sentences, focusing on the key facts, the main legal issue, and the final outcome. Ensure the summary is coherent, meaningful, and avoids repetition."
#     # },
#     # "medium": {
#     #     "max_new_tokens": 250,
#     #     "instruction": "Provide a 1-2 paragraph summary of this legal case, including the key facts, legal issues, main arguments, and the court's decision. Make sure the summary is logical, meaningful, and avoids repeated content."
#     # },
#     # "long": {
#     #     "max_new_tokens": 400,
#     #     "instruction": "Write a detailed summary of this legal case covering important facts, legal issues, arguments from both sides, judgment reasoning, and the final outcome. Ensure clarity, meaningfulness, and no redundant statements."
#     # },
#     # "very_long": {
#     #     "max_new_tokens": 1000,
#     #     "instruction": "Write a comprehensive and well-structured summary of this legal case, including all relevant details: facts, timeline of events, legal issues, arguments from both parties, reasoning behind the judgment, and the final outcome. Make the summary coherent, fully meaningful, and avoid any repetition."
#     # }
# #     mapping = {
# #     "short": {
# #         "max_new_tokens": 500,   # was 200
# #         "instruction": "Summarize this legal case clearly and concisely in 3-4 sentences, focusing on the key facts, the main legal issue, and the final outcome. Ensure the summary is coherent, meaningful, and avoids repetition."
# #     },
# #     "medium": {
# #         "max_new_tokens": 1500,  # was 500
# #         "instruction": "Provide a 1-2 paragraph summary of this legal case, including the key facts, legal issues, main arguments, and the court's decision. Make sure the summary is logical, meaningful, and avoids repeated content."
# #     },
# #     "long": {
# #         "max_new_tokens": 3000,  # was 1200
# #         "instruction": "Write a detailed summary of this legal case covering important facts, legal issues, arguments from both sides, judgment reasoning, and the final outcome. Ensure clarity, meaningfulness, and no redundant statements."
# #     },
# #     "very_long": {
# #         "max_new_tokens": 6000,  # was 2500
# #         "instruction": "Write a comprehensive and well-structured summary of this legal case, including all relevant details: facts, timeline of events, legal issues, arguments from both parties, reasoning behind the judgment, and the final outcome. Make the summary coherent, fully meaningful, and avoid any repetition."
# #     }
# # }
#     mapping = {
#     "short": {
#         "max_new_tokens": 500,   # was 200
#         "instruction": "Summarize this legal case in 3-4 sentences. Focus only on the primary legal issue, and the final outcome. Exclude detailed arguments and peripheral facts to keep it concise."
#     },
#     "medium": {
#         "max_new_tokens": 1500,  # was 500
#         "instruction": "Provide a 1-2 paragraph summary of this legal case. Include key facts, main legal issues, primary arguments from both sides, and the court's decision. Less critical details or minor facts can be omitted for brevity."
#     },
#     "long": {
#         "max_new_tokens": 3000,  # was 1200
#         "instruction": "Write a detailed summary of this legal case, covering important facts, all significant legal issues, arguments from both parties, reasoning behind the judgment, and the final outcome. Minor or redundant details can be lightly condensed."
#     },
#     "very_long": {
#         "max_new_tokens": 6000,  # was 2500
#         "instruction": "Write a comprehensive, well-structured summary of this legal case, including all relevant facts, timeline of events, legal issues, arguments from both parties, judgment reasoning, and final outcome. Include as many details as needed while avoiding repetition."
#     }
# }




#     cfg = mapping.get(level, mapping["short"])

#     # Simplified prompt format
#     prompt = f"{cfg['instruction']}\n\n{text}\n\nSummary:"

#     try:
#         # Tokenize with proper truncation
#         inputs = tokenizer(
#             prompt,
#             return_tensors="pt",
#             truncation=True,
#             max_length=2048  # Reduced to ensure room for generation
#         ).to(model.device)

#         # Generate response
#         with torch.no_grad():
#             outputs = model.generate(
#                 **inputs,
#                 max_new_tokens=cfg["max_new_tokens"],
#                 do_sample=True,
#                 temperature=0.7,
#                 top_p=0.9,
#                 eos_token_id=tokenizer.eos_token_id,
#                 pad_token_id=tokenizer.pad_token_id,
#                 repetition_penalty=1.1
#             )

#         # Extract only the generated text (after the prompt)
#         generated_tokens = outputs[0][inputs.input_ids.shape[-1]:]
#         summary = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

#         # Clean up any remaining prompt fragments
#         summary = re.sub(r'^(Summary|Instruction):\s*', '', summary, flags=re.IGNORECASE).strip()

#         return summary if summary else "Could not generate summary. The text might be too complex or too long."

#     except Exception as e:
#         print(f"Summarization error: {e}")
#         return f"Error generating summary: {str(e)}"

# # -------- Question Answering --------
# def ask_with_context(question, context, max_answer_tokens=150):
#     if not context.strip():
#         return "No context provided to answer the question."

#     # Simplified prompt
#     prompt = f"Answer this question based on the context:\n\nContext: {context}\n\nQuestion: {question}\n\nAnswer:"

#     try:
#         inputs = tokenizer(
#             prompt,
#             return_tensors="pt",
#             truncation=True,
#             max_length=2048
#         ).to(model.device)

#         with torch.no_grad():
#             outputs = model.generate(
#                 **inputs,
#                 max_new_tokens=max_answer_tokens,
#                 do_sample=True,
#                 temperature=0.7,
#                 top_p=0.9,
#                 eos_token_id=tokenizer.eos_token_id,
#                 pad_token_id=tokenizer.pad_token_id,
#                 repetition_penalty=1.1
#             )

#         # Extract only the generated text
#         generated_tokens = outputs[0][inputs.input_ids.shape[-1]:]
#         answer = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

#         # Clean up any remaining prompt fragments
#         answer = re.sub(r'^(Answer|Question|Context):\s*', '', answer, flags=re.IGNORECASE).strip()

#         # Name extraction for questions about names
#         if "name" in question.lower() or "who" in question.lower():
#             names = re.findall(r"[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*", answer)
#             if names:
#                 return list(set(names))

#         return answer if answer else "Could not generate an answer. The question might be too complex or the context insufficient."

#     except Exception as e:
#         print(f"QA error: {e}")
#         return f"Error generating answer: {str(e)}"

# # -------- FastAPI app --------
# app = FastAPI()

# # --- Summarize from PDF ---
# @app.post("/summarize/pdf")
# async def summarize_pdf(file: UploadFile, level: str = Form("short")):
#     with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp:
#         tmp.write(await file.read())
#         tmp_path = tmp.name
#     try:
#         text = extract_pdf_text(tmp_path)
#         if not text.strip():
#             return JSONResponse({"error": "PDF is empty or unreadable"}, status_code=400)
#         result = summarize_text(text, level)
#         return JSONResponse({"summary": result})
#     except Exception as e:
#         return JSONResponse({"error": f"Processing error: {str(e)}"}, status_code=500)
#     finally:
#         os.remove(tmp_path)

# # --- Summarize from Raw Text ---
# @app.post("/summarize/text")
# async def summarize_text_input(text: str = Form(...), level: str = Form("short")):
#     if not text.strip():
#         return JSONResponse({"error": "Text input is empty"}, status_code=400)
#     result = summarize_text(text, level)
#     return JSONResponse({"summary": result})

# # --- Ask Question on PDF ---
# @app.post("/ask/pdf")
# async def ask_pdf(file: UploadFile, question: str = Form(...)):
#     with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp:
#         tmp.write(await file.read())
#         tmp_path = tmp.name
#     try:
#         text = extract_pdf_text(tmp_path)
#         if not text.strip():
#             return JSONResponse({"error": "PDF is empty or unreadable"}, status_code=400)
#         result = ask_with_context(question, text)
#         return JSONResponse({"answer": result})
#     except Exception as e:
#         return JSONResponse({"error": f"Processing error: {str(e)}"}, status_code=500)
#     finally:
#         os.remove(tmp_path)

# # --- Ask Question on Raw Text ---
# @app.post("/ask/text")
# async def ask_text(question: str = Form(...), text: str = Form(...)):
#     if not text.strip():
#         return JSONResponse({"error": "Text input is empty"}, status_code=400)
#     result = ask_with_context(question, text)
#     return JSONResponse({"answer": result})

# # -------- Chat Function --------
# def chat_with_model(message, history=None, max_new_tokens=250):
#     """
#     Handles general chat (conversation-style) interaction with the model.
#     'history' can be a list of previous messages to maintain context.
#     """
#     if history is None:
#         history = []

#     # Combine history into the prompt
#     conversation = ""
#     for turn in history:
#         conversation += f"User: {turn['user']}\nAssistant: {turn['assistant']}\n"

#     prompt = f"{conversation}User: {message}\nAssistant:"

#     try:
#         inputs = tokenizer(
#             prompt,
#             return_tensors="pt",
#             truncation=True,
#             max_length=2048
#         ).to(model.device)

#         with torch.no_grad():
#             outputs = model.generate(
#                 **inputs,
#                 max_new_tokens=max_new_tokens,
#                 do_sample=True,
#                 temperature=0.8,
#                 top_p=0.9,
#                 eos_token_id=tokenizer.eos_token_id,
#                 pad_token_id=tokenizer.pad_token_id,
#                 repetition_penalty=1.1
#             )

#         generated_tokens = outputs[0][inputs.input_ids.shape[-1]:]
#         response = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
#         response = re.sub(r'^(Assistant|User):\s*', '', response, flags=re.IGNORECASE).strip()

#         return response if response else "I'm not sure how to respond to that."

#     except Exception as e:
#         print(f"Chat error: {e}")
#         return f"Error generating chat response: {str(e)}"


# # --- General Chat Endpoint ---
# @app.post("/chat/simple")
# async def chat_endpoint(message: str = Form(...)):
#     """
#     General-purpose chat endpoint.
#     Example usage: send a text message and receive a conversational reply.
#     """
#     if not message.strip():
#         return JSONResponse({"error": "Message is empty"}, status_code=400)

#     result = chat_with_model(message)
#     return JSONResponse({"reply": result})
from pypdf import PdfReader
import re, tempfile, os
from fastapi import FastAPI, UploadFile, Form
from fastapi.responses import JSONResponse
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# -------- PDF text extraction --------
def extract_pdf_text(pdf_path):
    try:
        reader = PdfReader(pdf_path)
        pages_text = []
        for page in reader.pages:
            txt = page.extract_text()
            if txt:
                pages_text.append(txt)
        return "\n\n".join(pages_text).strip()
    except Exception as e:
        print(f"PDF extraction error: {e}")
        return ""

# -------- Summarization --------
def summarize_text(text, level="short"):
    if not text.strip():
        return "No text content found to summarize."

    mapping = {
        "short": {
            "max_new_tokens": 500,
            "instruction": "Summarize this legal case in 3-4 sentences. Focus only on the primary legal issue, and the final outcome. Exclude detailed arguments and peripheral facts to keep it concise."
        },
        "medium": {
            "max_new_tokens": 1500,
            "instruction": "Provide a 1-2 paragraph summary of this legal case. Include key facts, main legal issues, primary arguments from both sides, and the court's decision. Less critical details or minor facts can be omitted for brevity."
        },
        "long": {
            "max_new_tokens": 3000,
            "instruction": "Write a detailed summary of this legal case, covering important facts, all significant legal issues, arguments from both parties, reasoning behind the judgment, and the final outcome. Minor or redundant details can be lightly condensed."
        },
        "very_long": {
            "max_new_tokens": 6000,
            "instruction": "Write a comprehensive, well-structured summary of this legal case, including all relevant facts, timeline of events, legal issues, arguments from both parties, judgment reasoning, and final outcome. Include as many details as needed while avoiding repetition."
        }
    }

    cfg = mapping.get(level, mapping["short"])

    # 🧠 LLaMA-compatible summarization prompt
    prompt = (
        f"[INST] <<SYS>> You are a legal AI assistant. "
        f"Summarize the document clearly and meaningfully according to the level of detail requested. "
        f"Avoid repetition and incomplete statements. <</SYS>>\n\n"
        f"Context:\n{text}\n\n"
        f"Instruction: {cfg['instruction']}\n[/INST]\nSummary:"
    )

    try:
        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=2048
        ).to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=cfg["max_new_tokens"],
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
                repetition_penalty=1.1
            )

        generated_tokens = outputs[0][inputs.input_ids.shape[-1]:]
        summary = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
        summary = re.sub(r'^(Summary|Instruction):\s*', '', summary, flags=re.IGNORECASE).strip()

        return summary if summary else "Could not generate summary. The text might be too complex or too long."

    except Exception as e:
        print(f"Summarization error: {e}")
        return f"Error generating summary: {str(e)}"


# -------- Question Answering --------
def ask_with_context(question, context, max_answer_tokens=150):
    if not context.strip():
        return "No context provided to answer the question."

    prompt = f"Answer this question based only on the context below.\nIf the answer is not explicitly found, say 'Not mentioned'.\n\nContext:\n{context}\n\nQuestion: {question}\nAnswer:"

    try:
        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=2048
        ).to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_answer_tokens,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
                repetition_penalty=1.1
            )

        generated_tokens = outputs[0][inputs.input_ids.shape[-1]:]
        answer = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

        # --- Clean unwanted parts ---
        # Remove extra Q/A sections if the model generated them
        answer = re.split(r'(?i)(question:|context:)', answer)[0].strip()

        # Keep only first sentence or short answer
        if len(answer.split('.')) > 1:
            first_sentence = answer.split('.')[0].strip()
            if len(first_sentence.split()) >= 3:
                answer = first_sentence

        # Extract only name if question asks for "who" or "author"
        if any(word in question.lower() for word in ["who", "author", "name"]):
            names = re.findall(r"[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*", answer)
            if names:
                return list(set(names))[0]  # Return first valid name

        return answer if answer else "Not mentioned"

    except Exception as e:
        print(f"QA error: {e}")
        return f"Error generating answer: {str(e)}"


# -------- FastAPI app --------
app = FastAPI()

# --- Summarize from PDF ---
@app.post("/summarize/pdf")
async def summarize_pdf(file: UploadFile, level: str = Form("short")):
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp:
        tmp.write(await file.read())
        tmp_path = tmp.name
    try:
        text = extract_pdf_text(tmp_path)
        if not text.strip():
            return JSONResponse({"error": "PDF is empty or unreadable"}, status_code=400)
        result = summarize_text(text, level)
        return JSONResponse({"summary": result})
    except Exception as e:
        return JSONResponse({"error": f"Processing error: {str(e)}"}, status_code=500)
    finally:
        os.remove(tmp_path)

# --- Summarize from Raw Text ---
@app.post("/summarize/text")
async def summarize_text_input(text: str = Form(...), level: str = Form("short")):
    if not text.strip():
        return JSONResponse({"error": "Text input is empty"}, status_code=400)
    result = summarize_text(text, level)
    return JSONResponse({"summary": result})

# --- Ask Question on PDF ---
@app.post("/ask/pdf")
async def ask_pdf(file: UploadFile, question: str = Form(...)):
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp:
        tmp.write(await file.read())
        tmp_path = tmp.name
    try:
        text = extract_pdf_text(tmp_path)
        if not text.strip():
            return JSONResponse({"error": "PDF is empty or unreadable"}, status_code=400)
        result = ask_with_context(question, text)
        return JSONResponse({"answer": result})
    except Exception as e:
        return JSONResponse({"error": f"Processing error: {str(e)}"}, status_code=500)
    finally:
        os.remove(tmp_path)

# --- Ask Question on Raw Text ---
@app.post("/ask/text")
async def ask_text(question: str = Form(...), text: str = Form(...)):
    if not text.strip():
        return JSONResponse({"error": "Text input is empty"}, status_code=400)
    result = ask_with_context(question, text)
    return JSONResponse({"answer": result})

# -------- Chat Function --------
# def chat_with_model(message, history=None, max_new_tokens=250):
#     if history is None:
#         history = []

#     conversation = ""
#     for turn in history:
#         conversation += f"User: {turn['user']}\nAssistant: {turn['assistant']}\n"

#     prompt = f"{conversation}User: {message}\nAssistant:"

#     try:
#         inputs = tokenizer(
#             prompt,
#             return_tensors="pt",
#             truncation=True,
#             max_length=2048
#         ).to(model.device)

#         with torch.no_grad():
#             outputs = model.generate(
#                 **inputs,
#                 max_new_tokens=max_new_tokens,
#                 do_sample=True,
#                 temperature=0.8,
#                 top_p=0.9,
#                 eos_token_id=tokenizer.eos_token_id,
#                 pad_token_id=tokenizer.pad_token_id,
#                 repetition_penalty=1.1
#             )

#         generated_tokens = outputs[0][inputs.input_ids.shape[-1]:]
#         response = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
#         response = re.sub(r'^(Assistant|User):\s*', '', response, flags=re.IGNORECASE).strip()

#         return response if response else "I'm not sure how to respond to that."

#     except Exception as e:
#         print(f"Chat error: {e}")
#         return f"Error generating chat response: {str(e)}"
def chat_with_model(message, max_new_tokens=120):

    prompt = (
        "You are an AI assistant. Give a direct, standalone answer to the question. "
        "Do NOT generate follow-up questions. Do NOT continue in Q&A format.\n\n"
        f"Question: {message}\nAnswer:"
    )

    try:
        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=2048
        ).to(model.device)

        input_len = inputs.input_ids.shape[-1]

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                min_new_tokens=30,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                repetition_penalty=1.1,
                no_repeat_ngram_size=4,
                pad_token_id=tokenizer.eos_token_id,
            )

        generated_tokens = outputs[0][input_len:]
        response = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

        # 🚫 HARD STOP if model tries to start new Question
        response = re.split(r'(?i)\b(question:|q:)\b', response)[0].strip()

        # Remove list-like Q&A patterns
        response = re.sub(r'\bAnswer:\s*$', '', response).strip()

        if len(response.split()) < 5:
            return "Model could not produce a reliable answer."

        return response

    except Exception as e:
        print(f"Chat error: {e}")
        return f"Error generating chat response: {str(e)}"

# --- General Chat Endpoint ---
@app.post("/chat/simple")
async def chat_endpoint(message: str = Form(...)):
    if not message.strip():
        return JSONResponse({"error": "Message is empty"}, status_code=400)
    result = chat_with_model(message)
    return JSONResponse({"reply": result})



In [ ]:
# from fastapi import FastAPI, UploadFile, Form
# from fastapi.responses import JSONResponse
# import tempfile, os

# # Initialize FastAPI app
# app = FastAPI()

# # PDF Summarization endpoint
# @app.post("/summarize/")
# async def summarize_pdf(file: UploadFile, level: str = Form("short")):
#     with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp:
#         tmp.write(await file.read())
#         tmp_path = tmp.name
#     try:
#         text = extract_pdf_text(tmp_path)
#         result = summarize_text(text, level)
#         return JSONResponse({"summary": result})
#     finally:
#         os.remove(tmp_path)

# # PDF Question Answering endpoint
# @app.post("/ask/")
# async def ask_pdf(file: UploadFile, question: str = Form(...)):
#     with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp:
#         tmp.write(await file.read())
#         tmp_path = tmp.name
#     try:
#         text = extract_pdf_text(tmp_path)
#         result = ask_with_context(question, text)
#         return JSONResponse({"answer": result})
#     finally:
#         os.remove(tmp_path)


In [ ]:
# Step 1: Set your ngrok token (run once)
!ngrok config add-authtoken 34bL9W89dmtsPnI1GV6bEMPUfZ6_2oBMTUFcibQLpeDeBbGd1

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import nest_asyncio
import uvicorn
from pyngrok import ngrok
import asyncio

nest_asyncio.apply()

public_url = ngrok.connect(8000)
print("🚀 Public URL:", public_url.public_url)

config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
server = uvicorn.Server(config)

await server.serve()   # ✅ Use await to run within notebook event loop

🚀 Public URL: https://subplacental-maryjo-polytonally.ngrok-free.dev


INFO:     Started server process [602]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     111.125.235.52:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     111.125.235.52:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     74.220.52.2:0 - "POST /chat/simple HTTP/1.1" 200 OK
